When use xlm-roberta. Restart the runtime after installing

In [ ]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.3 MB/s eta 0:00:00


In [2]:
import re
import random
import numpy as np
import pandas as pd

import spacy 

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer
from transformers import AutoModel, AutoTokenizer 

from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.metrics import confusion_matrix, classification_report,precision_score,accuracy_score,f1_score

import logging 
logging.basicConfig(level = logging.ERROR)




from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-2-4b184e2fc1c1>:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
#model_name = 'bert-base-multilingual-uncased'
model_name = 'bert-base-uncased'
#model_name = 'xlm-roberta-base'

OneStopEnglishCorpus

In [ ]:
#!pip install datasets
#from datasets import load_dataset

#dataset = load_dataset("onestop_english")

VikiWiki

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!wget https://www.dropbox.com/s/bmov3bgxv53lpu4/all_lang.csv?dl=0 -O all_lang.csv

--2023-04-18 09:41:47--  https://www.dropbox.com/s/bmov3bgxv53lpu4/all_lang.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/bmov3bgxv53lpu4/all_lang.csv [following]
--2023-04-18 09:41:48--  https://www.dropbox.com/s/raw/bmov3bgxv53lpu4/all_lang.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1fd101df4d97ac6321bb6fb9cb.dl.dropboxusercontent.com/cd/0/inline/B6ZDEkQnGURbfPIbg9xmQZYl7Kfd7SDRjqNhSwIQnmm9nvLY_Hp-nN9bhosPdHWx8hWeX87LaXTSOA2wLnsf9e3sq6KTxy8F4PkQv2P0lxPLyw0Mk11-5nPg2JD5Z8L4I3__Y2vMwYNik7TGJ8QVK2R-0hYZ1RffnurAepEzbdh3hA/file# [following]
--2023-04-18 09:41:48--  https://uc1fd101df4d97ac6321bb6fb9cb.dl.dropboxusercontent.com/cd/0/inline/B6ZDEkQnGURbfPIbg9xmQZYl7Kfd7SDRjqNhSwIQnmm9nvLY_Hp-nN9bhosPdHWx8hWeX87LaXT

In [ ]:
df = pd.read_csv('/content/all_lang.csv')
df['lang'].unique()

In [ ]:
lang='fr'

dataset = df[df['lang']==lang]
dataset = dataset.dropna()

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


mean tree height

In [9]:
en_nlp = spacy.load('en_core_web_sm')

In [10]:
def walk_tree(node, depth):
    if node.n_lefts + node.n_rights > 0:
        return max(walk_tree(child, depth + 1) for child in node.children)
    else:
        return depth

def get_mean_tree_height(text):
  depths = []
  sentences = sent_tokenize(text)
  for sentence in sentences:
    doc = en_nlp(sentence)
    depth = [walk_tree(sent.root, 0) for sent in doc.sents]
    depths.extend(depth)
  return np.mean(depths)

In [11]:
dataset['content'].iloc[0]

"When you see the word Amazon, what’s the first thing you think of – the world’s biggest forest, the longest river or the largest internet shop – and which do you think is most important? \nThese are the questions in a debate about the internet. Brazil and Peru have made objections to a bid made by the US online shop for the domain name, “.amazon”. \nAmazon has asked for its company name to be a top-level domain name (currently “.com”), but the South American governments say this would stop the use of this internet address for environmental protection, indigenous rights and other public interest uses. \nThere are many other disputed claims to names, including “.patagonia”. \nUntil now, the differences between commercial, governmental and other types of identity were easy to see in every internet address by the use of “.com”, “.gov” and 20 other categories. \nBut soon there are going to be more of these categories – or generic top-level domains (gTLDs) as they are technically known. \nT

In [12]:
get_mean_tree_height(dataset['content'].iloc[0])

6.190476190476191

In [13]:
get_mean_tree_height(dataset['content'].iloc[471])

6.666666666666667

In [14]:
dataset['mean_tree_height'] = dataset['content'].progress_apply(get_mean_tree_height)

  0%|          | 0/567 [00:00<?, ?it/s]

mean sentence length

In [15]:
sen_len = []
for i in dataset['content']:
  curr_sen_len = []
  for j in sent_tokenize(i):
    curr_sen_len.append(len(word_tokenize(j)))
  mean_len = np.mean(curr_sen_len)
  sen_len.append(mean_len)

In [16]:
sen_len[400:410]

[20.084745762711865,
 24.56756756756757,
 30.625,
 33.911764705882355,
 27.363636363636363,
 32.25,
 19.796610169491526,
 24.755102040816325,
 31.083333333333332,
 34.142857142857146]

In [17]:
sen_len[100:110]

[23.06896551724138,
 21.84375,
 21.75,
 20.625,
 22.785714285714285,
 20.12121212121212,
 24.304347826086957,
 18.32608695652174,
 19.9,
 23.296296296296298]

In [18]:
dataset['mean_sentence_len'] = sen_len

In [19]:
dataset

,content,labels,avg_nounphrases,mean_tree_height,mean_sentence_len
0,"When you see the word Amazon, what’s the first...",0,6.909091,6.190476,25.400000
1,"To tourists, Amsterdam still seems very libera...",0,6.869565,5.727273,24.666667
2,"Anitta, a music star from Brazil, has millions...",0,6.148148,5.384615,20.538462
3,Google has made maps of the world’s highest mo...,0,7.166667,5.826087,23.260870
4,The auction of a Banksy painting that disappea...,0,7.363636,6.047619,24.526316
...,...,...,...,...,...
562,"In typical bad-boyfriend style, Dan Sullivan w...",2,6.333333,4.771429,22.833333
563,Thousands of people protested on Australia’s b...,2,8.666667,5.931034,28.653846
564,1 Race engineer \nA race engineer liaises betw...,2,7.571429,6.023810,24.707317
565,More than one million British workers might be...,2,11.214286,7.653846,39.260870


In [ ]:
!pip install lingfeat

In [ ]:
from lingfeat import extractor

In [ ]:
def get_avg_nounphrase(text):
  LingFeat = extractor.pass_text(text)
  LingFeat.preprocess()
  avg = LingFeat.PhrF_()['as_NoPhr_C']
  return avg

In [ ]:
dataset['avg_nounphrases'] = dataset['content'].progress_apply(get_avg_nounphrase)

Dataset

In [20]:
class DiffDataset(Dataset):
    def __init__(self, contents, labels, tokenizer, max_len, feature_1=None, feature_2=None, feature_3=None):
        self.contents = contents
        self.labels = labels
        self.tokenizer = tokenizer

        self.feature_1 = feature_1
        self.feature_2 = feature_2
        self.feature_3 = feature_3

        self.max_len = max_len
  
    def __len__(self):
        return len(self.contents)
  
    def __getitem__(self, item):
        content = str(self.contents[item])
        labels = self.labels[item]

        feature_1 = self.feature_1[item]
        feature_2 = self.feature_2[item]
        feature_3 = self.feature_3[item]

        # Tokenizing the texts, while also including special tokens 
        # for start and end of the text, as well as padding
        encoding = self.tokenizer.encode_plus(
          content,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt', # We return here the data as Pytorch Tensor
        )

        return {
          'content': content,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(labels, dtype=torch.long), 
          'feature_1': torch.tensor([feature_1], dtype=torch.long),
          'feature_2': torch.tensor([feature_2], dtype=torch.long),
          'feature_3': torch.tensor([feature_3], dtype=torch.long)
        }


In [21]:
df_temp, df_test = train_test_split(dataset, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_temp, test_size=0.1, random_state=42)

print("size of training set : " + str(df_train.shape[0]))
print("size of validation set : " + str(df_val.shape[0]))
print("size of test set : " + str(df_test.shape[0]))



size of training set : 407
size of validation set : 46
size of test set : 114


In [22]:
def generate_dataloader(df, tokenizer, max_len, batch_size):
    ds = DiffDataset(
        contents=df.content.to_numpy(),
        labels=df.labels.to_numpy(),

        feature_1 = df.mean_sentence_len.to_numpy(),
        feature_2 = df.mean_tree_height.to_numpy(),
        feature_3 = df.avg_nounphrases.to_numpy(),
        
        
        tokenizer=tokenizer,
        max_len=max_len,
      )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
      )


In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [24]:
max_len = 128 
batch_size = 8

train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)

data = next(iter(train_dataloader))



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [25]:
class DiffClassifier(nn.Module):

    def __init__(self, n_classes):
        super(DiffClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        # Adding drop out, keeping 90% of the last neurons of the raw BERT model
        #self.drop = nn.Dropout(p=0.1)
        # The last linear layer for multiclass classification
        self.out = nn.Linear(self.bert.config.hidden_size+3, n_classes)
  
    # Forward propagation function
    def forward(self, input_ids, attention_mask, feature_1, feature_2, feature_3):
        model_outs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = model_outs[0]

        pooled_output = model_outs[1]
        pooled_output = torch.cat((pooled_output, feature_1, feature_2, feature_3) , dim=1)

        return self.out(pooled_output)


In [26]:
len(dataset['labels'].unique())

3

In [54]:
model = DiffClassifier(len(dataset['labels'].unique()))

# Running the classifier on GPU
model = model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
features = data['feature_1'].to(device)
print(input_ids.shape)
print(attention_mask.shape)
print(features.shape)

torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 1])


In [55]:
optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  weight_decay = 0.999,
                  correct_bias=False)
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0.09,
                                            num_training_steps=len(train_dataloader)*epochs)
loss_fn = nn.CrossEntropyLoss().to(device)


In [31]:
# Computing multiclass precision for the outputs of the model
def compute_precision(outputs, labels):
  op = outputs.cpu()
  la = labels.cpu()
  _, preds = torch.max(op, dim=1)
  # We choose 'weighted' averaging of the precision of each label because it takes into account the imbalance of labels in our tweets dataset
  # other viable averaging methods are 'micro'
  return torch.tensor(precision_score(la, preds, average='weighted',zero_division=0))

In [32]:
def train(model, train_dataloader,optimizer,scheduler):

        # Reset tracking variables at the beginning of each epoch
        precision, correct_predictions, batch_counts = 0, 0, 0
        losses = []

        # Put the model into the training mode
        model.train()

  
        # For each batch of training data...
        for d in train_dataloader:
              batch_counts +=1
              # Load batch to GPU
              input_ids = d["input_ids"].to(device)
              attention_mask = d["attention_mask"].to(device)
              labels = d["labels"].to(device)
              feature_1 = d["feature_1"].to(device)
              feature_2 = d["feature_2"].to(device)
              feature_3 = d["feature_3"].to(device)

              outputs = model(input_ids=input_ids,attention_mask=attention_mask, feature_1=feature_1, feature_2=feature_2, feature_3=feature_3)
              
              _, preds = torch.max(outputs, dim=1)
              
              # Compute loss and accumulate the loss values

              loss = loss_fn(outputs, labels)

              correct_predictions += torch.sum(preds == labels)
              losses.append(loss.item())
              precision +=  compute_precision(outputs, labels)
            
              loss.backward()

              # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
              # Update parameters and the learning rate  
              optimizer.step()
              scheduler.step()
              optimizer.zero_grad()
          # Accuracy, loss, precision
        return correct_predictions.double() / len(df_train), np.mean(losses), precision/batch_counts

In [34]:
def eval(model, valid_dataloader, loss_fn, device, n):
    model = model.eval()


    correct_predictions , precision ,batch_counts = 0,0,0
    losses = []

    with torch.no_grad():
        for d in valid_dataloader:
            batch_counts += 1

            # Preparing inputs
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            feature_1 = d["feature_1"].to(device)
            feature_2 = d["feature_2"].to(device)
            feature_3 = d["feature_3"].to(device)

            # Running inference using the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                feature_1=feature_1, 
                feature_2=feature_2,
                feature_3=feature_3
              )
            
            # Running softmax on the outputs
            _, preds = torch.max(outputs, dim=1)

            # Computing loss function
            loss = loss_fn(outputs, labels)

            # Counting the correct occurences
            correct_predictions += torch.sum(preds == labels)

            # Computing the precision (true positives/true positives + false positives) 
            # for each class and label, and find their average weighted by support 
            precision += compute_precision(outputs,labels)

            losses.append(loss.item())
    # Accuracy, loss, precision 
    return correct_predictions.double()/n, np.mean(losses), precision/batch_counts 

In [56]:
history = defaultdict(list)
b_accuracy = 0

for epoch in range(epochs):
  
  print(f'Epoch {epoch + 1}/{epochs}')
  print('-' * 20)

  train_acc, train_loss, train_preci = train(
        model,
        train_dataloader,    
        optimizer, 
        scheduler,
      )
  print(f"Train : Loss {train_loss}, Accuracy : {train_acc*100:.2f} %, Precision : {train_preci}")

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['train_precision'].append(train_preci)

  val_acc, val_loss, val_preci = eval(
        model,
        val_dataloader,
        loss_fn, 
        device,
        len(df_val),
      )

  print(f'Val : Loss :{val_loss}, Accuracy : {val_acc*100:.2f} %, Precision : {val_preci}')  
  print()

  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  history['val_precision'].append(val_preci)
  
  if val_acc > b_accuracy:
    torch.save(model.state_dict(), 'bert_model_baseline.bin')
    b_accuracy = val_acc

Epoch 1/5
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Train : Loss 0.8974175061665329, Accuracy : 52.83 %, Precision : 0.48478641456582633


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.7190704345703125, Accuracy : 63.04 %, Precision : 0.6331349206349207

Epoch 2/5
--------------------


...

Val : Loss :0.1854736547296246, Accuracy : 95.65 %, Precision : 0.9694444444444444

Epoch 5/5
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Train : Loss 0.0823687363333781, Accuracy : 97.54 %, Precision : 0.981045751633987


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.20092381490394473, Accuracy : 95.65 %, Precision : 0.9694444444444444



In [38]:
def testing(model, dataloader):
  model = model.eval()
  
  contents = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in dataloader:

      texts = d["content"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["labels"].to(device)
      feature_1 = d["feature_1"].to(device)
      feature_2 = d["feature_2"].to(device)
      feature_3 = d["feature_3"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask, 
        feature_1=feature_1, 
        feature_2=feature_2,
        feature_3=feature_3
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      contents.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return contents, predictions, prediction_probs, real_values

In [57]:
y_contents, y_pred, y_pred_probs, y_test = testing(
  model,
  test_dataloader
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
